# Import Libraries

In [1]:
import os

import pandas as pd
import numpy as np

from datetime import datetime

import sqlite3
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.orm import sessionmaker

import statsmodels.formula.api as sm
from statsmodels.tsa import tsatools
from dmba import regressionSummary

no display found. Using non-interactive Agg backend


# Establish Directory

In [2]:
os.getcwd()

'/Users/michaelbyrd/Desktop/MIS548'

In [3]:
os.chdir('/Users/michaelbyrd/Desktop/MIS548/Million Songs')

In [4]:
os.getcwd()

'/Users/michaelbyrd/Desktop/MIS548/Million Songs'

# Breakdown Function

In [5]:
# Function for Complete Breakdown
def breakdown(df):
    # print("Head", "\n", df.head())
    # print("Tail", "\n", df.tail())
    print("\n\n", "Columns", "\n", df.columns)
    print("\n\n", "Describe", "\n", df.describe)
    print("\n\n\n", "Correlation Matrix", "\n", df.corr())
    print("\n\n\n", "Info", "\n", df.info())
    print("\n\n\n", "Distinct Count", "\n", df.nunique())
    print("\n\n\n", "Count", "\n", df.count())
    print("\n\n\n", "DTypes", "\n", df.dtypes)
    print("\n\n\n", "Shape", "\n", df.shape)
    print("\n\n\n", "Nulls", "\n", df.isna().sum())
    print("\n\n\n", "Null %", "\n", round(df.isna().sum()/len(df)*100, 2))

# SQLite

## Track MetaData

In [6]:
tm_con = sqlite3.connect("track_metadata.db")

In [7]:
tm_cur = tm_con.cursor()

In [8]:
# Query to fetch table names from the master table
tm_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = tm_cur.fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
songs


In [9]:
# Select all data from the table
songs_df = pd.read_sql('SELECT * FROM songs', tm_con)
songs_df

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRYYYUS12903CD2DF0,O Samba Da Vida,SOTXAME12AB018F136,Pacha V.I.P.,AR7Z4J81187FB3FC59,9d50cb20-7e42-45cc-b0dd-154c3e92a577,Kiko Navarro,217.44281,0.528617,0.411595,0,7522478,-1,0
999996,TRYYYJO128F426DA37,Jago Chhadeo,SOXQYIQ12A8C137FBB,Naale Baba Lassi Pee Gya,ART5FZD1187B9A7FCF,2357c400-9109-42b6-b3fe-9e2d9f8e3872,Kuldeep Manak,244.16608,0.401500,0.374866,0,1632096,-1,0
999997,TRYYYMG128F4260ECA,Novemba,SOHODZI12A8C137BB3,Dub_Connected: electronic music,ARZ3R6M1187B9AF750,8b97e9c8-61f5-4615-9a96-276f24204e34,Gabriel Le Mar,553.03791,0.556918,0.336914,0,2219291,-1,0
999998,TRYYYDJ128F9310A21,Faraday,SOLXGOR12A81C21EB7,The Trance Collection Vol. 2,ARCMCOK1187B9B1073,4ac5f3de-c5ad-475e-ad50-41f1ef9dba20,Elude,484.51873,0.403668,0.256935,0,5472456,-1,0


In [10]:
breakdown(songs_df)



 Columns 
 Index(['track_id', 'title', 'song_id', 'release', 'artist_id', 'artist_mbid',
       'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
       'year', 'track_7digitalid', 'shs_perf', 'shs_work'],
      dtype='object')


 Describe 
 <bound method NDFrame.describe of                   track_id                                title  \
0       TRMMMYQ128F932D901                         Silent Night   
1       TRMMMKD128F425225D                          Tanssi vaan   
2       TRMMMRX128F93187D9                    No One Could Ever   
3       TRMMMCH128F425532C                        Si Vos Querés   
4       TRMMMWA128F426B589                     Tangle Of Aspens   
...                    ...                                  ...   
999995  TRYYYUS12903CD2DF0                      O Samba Da Vida   
999996  TRYYYJO128F426DA37                         Jago Chhadeo   
999997  TRYYYMG128F4260ECA                              Novemba   
999998  TRYYYDJ128F9310A21      

In [11]:
songs_df.to_csv('track_metadata.csv', index = False)

In [12]:
tm_con.close()

## Artists Similarity

In [13]:
as_con = sqlite3.connect("artist_similarity.db")

as_cur = as_con.cursor()

In [14]:
# Query to fetch table names from the master table
as_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = as_cur.fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(table[0])
    


Tables in the database:
artists
similarity


In [15]:
# Select all data from the table
artists_df = pd.read_sql('SELECT * FROM artists', as_con)
artists_df

,artist_id
0,AR002UA1187B9A637D
1,AR003FB1187B994355
2,AR006821187FB5192B
3,AR009211187B989185
4,AR009SZ1187B9A73F4
...,...
44740,ARZZXT51187FB4627E
44741,ARZZYRB1187B99D0B6
44742,ARZZYRH11C8A416A12
44743,ARZZZAI124207819C9


In [16]:
breakdown(artists_df)



 Columns 
 Index(['artist_id'], dtype='object')


 Describe 
 <bound method NDFrame.describe of                 artist_id
0      AR002UA1187B9A637D
1      AR003FB1187B994355
2      AR006821187FB5192B
3      AR009211187B989185
4      AR009SZ1187B9A73F4
...                   ...
44740  ARZZXT51187FB4627E
44741  ARZZYRB1187B99D0B6
44742  ARZZYRH11C8A416A12
44743  ARZZZAI124207819C9
44744  ARZZZKG1271F573BC4

[44745 rows x 1 columns]>



 Correlation Matrix 
 Empty DataFrame
Columns: []
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44745 entries, 0 to 44744
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   artist_id  44745 non-null  object
dtypes: object(1)
memory usage: 349.7+ KB



 Info 
 None



 Distinct Count 
 artist_id    44745
dtype: int64



 Count 
 artist_id    44745
dtype: int64



 DTypes 
 artist_id    object
dtype: object



 Shape 
 (44745, 1)



 Nulls 
 artist_id    0
dtype: int64



 Null %

In [17]:
# Select all data from the table
similarity_df = pd.read_sql('SELECT * FROM similarity', as_con)
similarity_df 

,target,similar
0,AR002UA1187B9A637D,ARQDOR81187FB3B06C
1,AR002UA1187B9A637D,AROHMXJ1187B989023
2,AR002UA1187B9A637D,ARAGWVR1187B9B749B
3,AR002UA1187B9A637D,AREQVWS1241B9CC0A4
4,AR002UA1187B9A637D,ARHBE351187FB3B0CD
...,...,...
2201911,ARZZZKG1271F573BC4,ARBYTEZ1187FB42EC2
2201912,ARZZZKG1271F573BC4,AR3B4IC1187B98C8B6
2201913,ARZZZKG1271F573BC4,ARM9KFI1187B9B356C
2201914,ARZZZKG1271F573BC4,ARF5QVT1187B9946B9


In [18]:
breakdown(similarity_df)



 Columns 
 Index(['target', 'similar'], dtype='object')


 Describe 
 <bound method NDFrame.describe of                      target             similar
0        AR002UA1187B9A637D  ARQDOR81187FB3B06C
1        AR002UA1187B9A637D  AROHMXJ1187B989023
2        AR002UA1187B9A637D  ARAGWVR1187B9B749B
3        AR002UA1187B9A637D  AREQVWS1241B9CC0A4
4        AR002UA1187B9A637D  ARHBE351187FB3B0CD
...                     ...                 ...
2201911  ARZZZKG1271F573BC4  ARBYTEZ1187FB42EC2
2201912  ARZZZKG1271F573BC4  AR3B4IC1187B98C8B6
2201913  ARZZZKG1271F573BC4  ARM9KFI1187B9B356C
2201914  ARZZZKG1271F573BC4  ARF5QVT1187B9946B9
2201915  ARZZZKG1271F573BC4  ARHTAPO1187FB5AADE

[2201916 rows x 2 columns]>



 Correlation Matrix 
 Empty DataFrame
Columns: []
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201916 entries, 0 to 2201915
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   target   object
 1   similar  object
dtypes: object(2)
memory usage

In [19]:
artists_df.to_csv('artists.csv', index = False)
similarity_df .to_csv('similarity.csv', index = False)

In [20]:
as_con.close()

## Artists Term

In [21]:
at_con = sqlite3.connect("artist_term.db")

at_cur = at_con.cursor()

In [22]:
# Query to fetch table names from the master table
at_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = at_cur.fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(table[0])
    


Tables in the database:
artists
terms
artist_term
mbtags
artist_mbtag


In [23]:
df_artists1 = pd.read_sql('SELECT * FROM artists', at_con)
df_artists1

,artist_id
0,AR002UA1187B9A637D
1,AR003FB1187B994355
2,AR006821187FB5192B
3,AR009211187B989185
4,AR009SZ1187B9A73F4
...,...
44740,ARZZXT51187FB4627E
44741,ARZZYRB1187B99D0B6
44742,ARZZYRH11C8A416A12
44743,ARZZZAI124207819C9


In [24]:
breakdown(df_artists1)



 Columns 
 Index(['artist_id'], dtype='object')


 Describe 
 <bound method NDFrame.describe of                 artist_id
0      AR002UA1187B9A637D
1      AR003FB1187B994355
2      AR006821187FB5192B
3      AR009211187B989185
4      AR009SZ1187B9A73F4
...                   ...
44740  ARZZXT51187FB4627E
44741  ARZZYRB1187B99D0B6
44742  ARZZYRH11C8A416A12
44743  ARZZZAI124207819C9
44744  ARZZZKG1271F573BC4

[44745 rows x 1 columns]>



 Correlation Matrix 
 Empty DataFrame
Columns: []
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44745 entries, 0 to 44744
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   artist_id  44745 non-null  object
dtypes: object(1)
memory usage: 349.7+ KB



 Info 
 None



 Distinct Count 
 artist_id    44745
dtype: int64



 Count 
 artist_id    44745
dtype: int64



 DTypes 
 artist_id    object
dtype: object



 Shape 
 (44745, 1)



 Nulls 
 artist_id    0
dtype: int64



 Null %

In [25]:
df_terms = pd.read_sql('SELECT * FROM terms', at_con)
df_terms

,term
0,00s
1,00s alternative
2,00s country
3,00s indie
4,00s pop
...,...
7638,zxzw 2006
7639,zxzw 2007
7640,zxzw 2008
7641,zxzw 2009


In [26]:
breakdown(df_terms)



 Columns 
 Index(['term'], dtype='object')


 Describe 
 <bound method NDFrame.describe of                  term
0                 00s
1     00s alternative
2         00s country
3           00s indie
4             00s pop
...               ...
7638        zxzw 2006
7639        zxzw 2007
7640        zxzw 2008
7641        zxzw 2009
7642           zydeco

[7643 rows x 1 columns]>



 Correlation Matrix 
 Empty DataFrame
Columns: []
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7643 entries, 0 to 7642
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   term    7643 non-null   object
dtypes: object(1)
memory usage: 59.8+ KB



 Info 
 None



 Distinct Count 
 term    7643
dtype: int64



 Count 
 term    7643
dtype: int64



 DTypes 
 term    object
dtype: object



 Shape 
 (7643, 1)



 Nulls 
 term    0
dtype: int64



 Null % 
 term    0.0
dtype: float64


In [27]:
df_artist_term = pd.read_sql('SELECT * FROM artist_term', at_con)
df_artist_term

,artist_id,term
0,AR002UA1187B9A637D,garage rock
1,AR002UA1187B9A637D,country rock
2,AR002UA1187B9A637D,free jazz
3,AR002UA1187B9A637D,oi
4,AR002UA1187B9A637D,space rock
...,...,...
1109376,ARZZZKG1271F573BC4,heavy
1109377,ARZZZKG1271F573BC4,dark
1109378,ARZZZKG1271F573BC4,instrumental
1109379,ARZZZKG1271F573BC4,beautiful


In [28]:
breakdown(df_artist_term)



 Columns 
 Index(['artist_id', 'term'], dtype='object')


 Describe 
 <bound method NDFrame.describe of                   artist_id          term
0        AR002UA1187B9A637D   garage rock
1        AR002UA1187B9A637D  country rock
2        AR002UA1187B9A637D     free jazz
3        AR002UA1187B9A637D            oi
4        AR002UA1187B9A637D    space rock
...                     ...           ...
1109376  ARZZZKG1271F573BC4         heavy
1109377  ARZZZKG1271F573BC4          dark
1109378  ARZZZKG1271F573BC4  instrumental
1109379  ARZZZKG1271F573BC4     beautiful
1109380  ARZZZKG1271F573BC4        guitar

[1109381 rows x 2 columns]>



 Correlation Matrix 
 Empty DataFrame
Columns: []
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109381 entries, 0 to 1109380
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   artist_id  1109381 non-null  object
 1   term       1109381 non-null  object
dtypes: object(2)
memo

In [29]:
df_mbtags = pd.read_sql('SELECT * FROM mbtags', at_con)
df_mbtags

,mbtag
0,00s
1,00s 10s
2,1 13 165900 150 7672 22647 34612 48720 59280 7...
3,1 7 186240 183 23558 41608 89158 111733 150833...
4,10s
...,...
2316,ਭੰਗੜਾ
2317,ハロー プロジェクト
2318,姚姓
2319,日本


In [30]:
breakdown(df_mbtags)



 Columns 
 Index(['mbtag'], dtype='object')


 Describe 
 <bound method NDFrame.describe of                                                   mbtag
0                                                   00s
1                                               00s 10s
2     1 13 165900 150 7672 22647 34612 48720 59280 7...
3     1 7 186240 183 23558 41608 89158 111733 150833...
4                                                   10s
...                                                 ...
2316                                              ਭੰਗੜਾ
2317                                         ハロー プロジェクト
2318                                                 姚姓
2319                                                 日本
2320                                               香港歌手

[2321 rows x 1 columns]>



 Correlation Matrix 
 Empty DataFrame
Columns: []
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2321 entries, 0 to 2320
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---

In [31]:
df_artist_mbtag = pd.read_sql('SELECT * FROM artist_mbtag', at_con)
df_artist_mbtag

,artist_id,mbtag
0,AR002UA1187B9A637D,uk
1,AR002UA1187B9A637D,rock
2,AR002UA1187B9A637D,garage rock
3,AR006821187FB5192B,bass
4,AR00A6H1187FB5402A,detroit
...,...,...
24772,ARZZMWO1187B9AFB36,usa
24773,ARZZMWO1187B9AFB36,metal
24774,ARZZXT51187FB4627E,classic pop and rock
24775,ARZZYRB1187B99D0B6,jazz


In [32]:
breakdown(df_artist_mbtag)



 Columns 
 Index(['artist_id', 'mbtag'], dtype='object')


 Describe 
 <bound method NDFrame.describe of                 artist_id                 mbtag
0      AR002UA1187B9A637D                    uk
1      AR002UA1187B9A637D                  rock
2      AR002UA1187B9A637D           garage rock
3      AR006821187FB5192B                  bass
4      AR00A6H1187FB5402A               detroit
...                   ...                   ...
24772  ARZZMWO1187B9AFB36                   usa
24773  ARZZMWO1187B9AFB36                 metal
24774  ARZZXT51187FB4627E  classic pop and rock
24775  ARZZYRB1187B99D0B6                  jazz
24776  ARZZYRB1187B99D0B6              canadian

[24777 rows x 2 columns]>



 Correlation Matrix 
 Empty DataFrame
Columns: []
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24777 entries, 0 to 24776
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   artist_id  24777 non-null  object
 1

In [33]:
"""
Tables in the database:
artists
terms
artist_term
mbtags
artist_mbtag
"""

df_artists1.to_csv('artists1.csv', index = False)
df_terms.to_csv('terms.csv', index = False)
df_artist_term.to_csv('artist_term.csv', index = False)
df_mbtags.to_csv('mbtags.csv', index = False)
df_artist_mbtag.to_csv('artist_mbtag.csv', index = False)

In [34]:
at_con.close()

## DataFrames

In [36]:
%who DataFrame

artists_df	 df_artist_mbtag	 df_artist_term	 df_artists1	 df_mbtags	 df_terms	 similarity_df	 songs_df	 


In [41]:
artists_df.head(3)

,artist_id
0,AR002UA1187B9A637D
1,AR003FB1187B994355
2,AR006821187FB5192B


In [42]:
df_artist_mbtag.head(3)

,artist_id,mbtag
0,AR002UA1187B9A637D,uk
1,AR002UA1187B9A637D,rock
2,AR002UA1187B9A637D,garage rock


In [43]:
df_artist_term.head(3)

,artist_id,term
0,AR002UA1187B9A637D,garage rock
1,AR002UA1187B9A637D,country rock
2,AR002UA1187B9A637D,free jazz


In [40]:
# Don't Need
df_artists1.head(3)

,artist_id
0,AR002UA1187B9A637D
1,AR003FB1187B994355
2,AR006821187FB5192B


In [44]:
df_mbtags.head(3)

,mbtag
0,00s
1,00s 10s
2,1 13 165900 150 7672 22647 34612 48720 59280 7...


In [45]:
df_terms.head(3)

,term
0,00s
1,00s alternative
2,00s country


In [46]:
similarity_df.head(3)

,target,similar
0,AR002UA1187B9A637D,ARQDOR81187FB3B06C
1,AR002UA1187B9A637D,AROHMXJ1187B989023
2,AR002UA1187B9A637D,ARAGWVR1187B9B749B


In [47]:
songs_df.head(3)

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0


# Dead Code